<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/GRU_Based_Text_Generator(pract).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from tensorflow.keras.layers import GRU, Dense, Embedding

In [ ]:
!unzip '/content/archive (5).zip'

Archive:  /content/archive (5).zip
replace ptb.train.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
text=pd.read_csv('/content/ptb.train.txt')

In [ ]:
tokenizer = Tokenizer(char_level=True)

In [ ]:
tokenizer.fit_on_texts(text)

In [ ]:
char_index = tokenizer.word_index
index_char = tokenizer.index_word
vocab_size = len(char_index) + 1

In [ ]:
sequences=tokenizer.texts_to_sequences(text)

In [ ]:
sequences = [item for sublist in sequences for item in sublist]

In [ ]:
seq_length = 40
X, y = [], []
for i in range(len(sequences) - seq_length):
    X.append(sequences[i:i + seq_length])
    y.append(sequences[i + seq_length])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length),
    GRU(150, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.0399 - loss: 3.2952
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1363 - loss: 3.2771
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1460 - loss: 3.2590
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1480 - loss: 3.2332
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.1382 - loss: 3.1955
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.1343 - loss: 3.1253
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1402 - loss: 3.0235
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.1343 - loss: 3.0155
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1245 - loss: 3.0048
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.1421 - loss: 2.9751 


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 40, 50)         │         1,350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 150)            │        90,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 27)             │         4,077 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 288,983 (1.10 MB)

 Trainable params: 96,327 (376.28 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 192,656 (752.57 KB)

In [ ]:
# Step 3: Text generation function
def generate_text(seed_text, num_chars=200):
    for _ in range(num_chars):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, truncating='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_char_index = np.argmax(predicted_probs)
        predicted_char = index_char[predicted_char_index]
        seed_text += predicted_char
    return seed_text

In [ ]:
def gradio_generate(seed_text, num_chars):
    return generate_text(seed_text, int(num_chars))

In [ ]:
interface = gr.Interface(
    fn=gradio_generate,
    inputs=[
        gr.Textbox(label="Seed Text", placeholder="Enter seed text, e.g., 'Once upon a time'"),
        gr.Slider(minimum=50, maximum=500, step=10, value=200, label="Number of Characters")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="GRU-Based Text Generator",
    description="Enter a seed text and specify the number of characters to generate new text using a GRU-based neural network."
)

In [ ]:
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://188af82fb56edbb836.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
